# polarity

## Setup

In [ ]:
# If necessary:
import os
os.remove('snorkel.db')

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import os
import sys
sys.path.append(os.environ['SNORKELHOME'] + '/tutorials/tables/')

## Parsing

In [ ]:
from snorkel import SnorkelSession
session = SnorkelSession()

### Define Parser

In [ ]:
import os
from snorkel.parser import CorpusParser, HTMLParser, OmniParser
from snorkel.utils import get_ORM_instance
from snorkel.queries import split_corpus

docs_path = os.environ['SNORKELHOME'] + '/tutorials/tables/data/hardware/hardware100_html/'
doc_parser = HTMLParser(path=docs_path)
context_parser = OmniParser()
cp = CorpusParser(doc_parser, context_parser, max_docs=100) 

### Run Parser

In [ ]:
%time corpus = cp.parse_corpus(name='Hardware', session=session)

session.add(corpus)
session.commit()

### Split Corpus

In [ ]:
from snorkel.models import Corpus

corpus = get_ORM_instance(Corpus, session, 'Hardware')
split_corpus(session, corpus, train=0.8, development=0.2, test=0, seed=3)

In [ ]:
# If necessary
import os
os.system('cp snorkel.db snorkel.db\ corpus');

## Extraction

In [ ]:
# If necessary:
import os
os.remove('snorkel.db');
os.system('cp snorkel.db\ corpus snorkel.db');

import sys
sys.path.append(os.environ['SNORKELHOME'] + '/tutorials/tables/')

from snorkel import SnorkelSession
session = SnorkelSession()

In [ ]:
from snorkel.models import candidate_subclass

Part_Polarity = candidate_subclass('Part_Polarity', ['part','polarity'])

### Define Matchers

In [ ]:
from snorkel.matchers import RegexMatchSpan, Union

eeca_matcher = RegexMatchSpan(rgx='([b]{1}[abcdefklnpqruyz]{1}[\swxyz]?[0-9]{3,5}[\s]?[A-Z\/]{0,5}[0-9]?[A-Z]?([-][A-Z0-9]{1,7})?([-][A-Z0-9]{1,2})?)')
jedec_matcher = RegexMatchSpan(rgx='([123]N\d{3,4}[A-Z]{0,5}[0-9]?[A-Z]?)')
jis_matcher = RegexMatchSpan(rgx='(2S[abcdefghjkmqrstvz]{1}[\d]{2,4})')
others_matcher = RegexMatchSpan(rgx='((NSVBC|SMBT|MJ|MJE|MPS|MRF|RCA|TIP|ZTX|ZT|TIS|TIPL|DTC|MMBT|PZT){1}[\d]{2,4}[A-Z]{0,3}([-][A-Z0-9]{0,6})?([-][A-Z0-9]{0,1})?)')
parts_matcher = Union(eeca_matcher, jedec_matcher, jis_matcher, others_matcher)

polarity_matcher = RegexMatchSpan(rgx=r'(pnp|npn)', ignore_case=True)

### Define ContextSpaces

In [ ]:
from snorkel.candidates import OmniNgrams
from hardware_utils import OmniNgramsPart, get_gold_dict
from collections import defaultdict
import os

# Make parts list
gold_file = os.environ['SNORKELHOME'] + '/tutorials/tables/data/hardware/hardware_gold.csv'
gold_parts = get_gold_dict(gold_file, doc_on=True, part_on=True, val_on=False)
parts_by_doc = defaultdict(set)
for part in gold_parts:
    parts_by_doc[part[0]].add(part[1])

part_ngrams = OmniNgramsPart(n_max=3, parts_by_doc=parts_by_doc)
polarity_ngrams = OmniNgrams(n_max=1)

### Run CandidateExtractor

In [ ]:
from snorkel.models import Corpus
from snorkel.candidates import CandidateExtractor
from snorkel.utils import get_ORM_instance
ce = CandidateExtractor(Part_Polarity, 
                        [part_ngrams, polarity_ngrams], 
                        [parts_matcher, polarity_matcher], 
                        throttler=None)

for corpus_name in ['Hardware Training', 'Hardware Development']:
    corpus = get_ORM_instance(Corpus, session, corpus_name)
    print "Extracting Candidates from %s" % corpus
    %time candidates = ce.extract(\
        corpus.documents, corpus_name + ' Candidates', session)
    session.add(candidates)
    print "%s contains %d Candidates" % (candidates, len(candidates))
session.commit()

### Assess Recall

In [ ]:
from snorkel.models import Candidate
from hardware_utils import entity_level_total_recall

all_candidates = session.query(Candidate).all()
gold_file = os.environ['SNORKELHOME'] + '/tutorials/tables/data/hardware/hardware_gold.csv'
(tp, fp, fn) = entity_level_total_recall(
    all_candidates, gold_file, 'polarity', relation=True)

In [ ]:
from pprint import pprint
pprint(fn)
print len(fn)

In [ ]:
# If necessary
import os
os.system('cp snorkel.db snorkel.db\ candidates');

## Gold Labels

In [1]:
# If necessary
import os
os.remove('snorkel.db');
os.system('cp snorkel.db\ candidates snorkel.db');

import sys
sys.path.append(os.environ['SNORKELHOME'] + '/tutorials/tables/')

from snorkel import SnorkelSession
session = SnorkelSession()

from snorkel.models import candidate_subclass
Part_Polarity = candidate_subclass('Part_Polarity', ['part','polarity'])

In [2]:
import os
from snorkel.models import CandidateSet
from hardware_utils import load_hardware_labels

gold_file = os.environ['SNORKELHOME'] + '/tutorials/tables/data/hardware/hardware_gold.csv'
for set_name in ['Training', 'Development']:
    candidate_set_name = 'Hardware %s Candidates' % set_name
    candidates = session.query(CandidateSet).filter(
        CandidateSet.name == candidate_set_name).one()
    label_set_name = 'Hardware %s Candidates -- Gold' % set_name
    annotation_key_name = 'Hardware %s Labels -- Gold' % set_name
    %time gold_candidates, annotation_key = load_hardware_labels(session,\
                           label_set_name, \
                           annotation_key_name, \
                           candidates, \
                           gold_file, \
                           'polarity')
    candidates_gold = session.query(CandidateSet).filter(
        CandidateSet.name == candidate_set_name + ' -- Gold').one()
    print "%d/%d Candidates in %s have positive Labels" % (
        len(candidates_gold), len(candidates), candidates)

Loading 92047 candidate labels
[========================================] 100%
CPU times: user 3min 41s, sys: 1.92 s, total: 3min 43s
Wall time: 3min 48s
86466/92047 Candidates in Candidate Set (Hardware Training Candidates) have positive Labels
Loading 22020 candidate labels
[========================================] 100%
CPU times: user 55.2 s, sys: 397 ms, total: 55.6 s
Wall time: 56.2 s
19634/22020 Candidates in Candidate Set (Hardware Development Candidates) have positive Labels


In [3]:
# If necessary
import os
os.system('cp snorkel.db snorkel.db\ labels');

### Assess Baseline Performance

In [17]:
from snorkel.models import Corpus, Candidate
from snorkel.utils import get_ORM_instance
from snorkel.lf_helpers import *
from hardware_utils import baseline, entity_level_f1
from itertools import chain

def LF_all_true(c):
    return True

def LF_complement_nearby(c):
    return 1 if set(['complement','complementary']).isdisjoint(
        chain.from_iterable(
            [get_phrase_ngrams(c.part), 
             get_phrase_ngrams(c.polarity),
             get_neighbor_phrase_ngrams(c.polarity)])) else 0

candidates = session.query(Candidate).all()
(tp, fp, tn, fn) = baseline(candidates, LF_complement_nearby)

corpus = get_ORM_instance(Corpus, session, 'Hardware')
gold_file = os.environ['SNORKELHOME'] + '/tutorials/tables/data/hardware/hardware_gold.csv'
(TP, FP, FN) = entity_level_f1(tp, fp, tn, fn, gold_file, corpus, 'polarity')

[========================================] 100%
Scoring on Entity-Level Gold Data
Corpus Precision 1.0
Corpus Recall    0.964
Corpus F1        0.982
----------------------------------------
TP: 801 | FP: 0 | FN: 30



In [18]:
from pprint import pprint
pprint(FP)

set([])


In [19]:
pprint(FN)

set([('CSEMS03485-1', '2N3905', 'PNP'),
     ('CSEMS03485-1', '2N3906', 'PNP'),
     ('SAMSD00049-518', 'BC546', 'NPN'),
     ('SAMSD00049-518', 'BC546A', 'NPN'),
     ('SAMSD00049-518', 'BC546B', 'NPN'),
     ('SAMSD00049-518', 'BC546C', 'NPN'),
     ('SAMSD00049-518', 'BC547', 'NPN'),
     ('SAMSD00049-518', 'BC547A', 'NPN'),
     ('SAMSD00049-518', 'BC547B', 'NPN'),
     ('SAMSD00049-518', 'BC547C', 'NPN'),
     ('SAMSD00049-518', 'BC548', 'NPN'),
     ('SAMSD00049-518', 'BC548A', 'NPN'),
     ('SAMSD00049-518', 'BC548B', 'NPN'),
     ('SAMSD00049-518', 'BC548C', 'NPN'),
     ('SAMSD00049-518', 'BC549', 'NPN'),
     ('SAMSD00049-518', 'BC549A', 'NPN'),
     ('SAMSD00049-518', 'BC549B', 'NPN'),
     ('SAMSD00049-518', 'BC549C', 'NPN'),
     ('SAMSD00049-518', 'BC550', 'NPN'),
     ('SAMSD00049-518', 'BC550A', 'NPN'),
     ('SAMSD00049-518', 'BC550B', 'NPN'),
     ('SAMSD00049-518', 'BC550C', 'NPN'),
     ('UTCLS02155-1', 'BC807', 'PNP'),
     ('UTCLS02155-1', 'BC807-16', 'PNP'),
    

In [7]:
from hardware_utils import entity_to_candidates

print len(candidates)
matches = entity_to_candidates((u'SIEMS01215-1', u'BC856', u'NPN'), candidates)
c = matches[0]
print c

114067
Part_Polarity(ImplicitSpan("BC856", parent=104141, words=[0,0], position=[2]), Span("NPN", parent=22719, chars=[16,18], words=[6,6]))


In [22]:
from snorkel.models import ImplicitSpan
from snorkel.lf_helpers import *
phrases = []
print list(get_neighbor_phrase_ngrams(c.polarity))
print c.polarity.parent
print phrases
print "SPAN_TYPE_[%s]" % ('IMPLICIT' if isinstance(c.polarity, ImplicitSpan) else 'EXPLICIT')
# list(get_phrase_ngrams(c.polarity))

[u'complementary', u'types', u':', u'bc', u'846', u',', u'bc', u'847', u',', u'type']
Phrase(Doc: SIEMS01215-1, Table: X, Row: X, Col: X, Position: 0, Text: BC 849, BC 850 (NPN))
[]
SPAN_TYPE_[EXPLICIT]


## Features

In [1]:
# If necessary:
import os
os.remove('snorkel.db');
os.system('cp snorkel.db\ labels snorkel.db');

from snorkel import SnorkelSession
session = SnorkelSession()

from snorkel.models import candidate_subclass
Part_Polarity = candidate_subclass('Part_Polarity', ['part','polarity'])

### Extract Features

In [2]:
from snorkel.models import CandidateSet
from snorkel.fast_annotations import FeatureManager
from snorkel.utils import get_ORM_instance

train = get_ORM_instance(CandidateSet, session, 'Hardware Training Candidates')
dev   = get_ORM_instance(CandidateSet, session, 'Hardware Development Candidates')

feature_manager = FeatureManager()
%time F_train = feature_manager.create(session, train, 'Train Features')
%time F_dev = feature_manager.update(session, dev, 'Train Features', expand_key_set=False)

[========================================] 100%
Bulk upserting 18469929 annotations...
Done.
Loading sparse Feature matrix...
CPU times: user 38min 50s, sys: 15min 51s, total: 54min 42s
Wall time: 59min 49s
[========================================] 100%
Bulk upserting 3671296 annotations...
Done.
Loading sparse Feature matrix...
CPU times: user 9min 21s, sys: 3min 24s, total: 12min 45s
Wall time: 13min 45s


In [3]:
# If necessary:
import os
os.system('cp snorkel.db snorkel.db\ featurized');

## LFs

In [1]:
# If necessary
import os
os.remove('snorkel.db');
os.system('cp snorkel.db\ featurized snorkel.db');

from snorkel import SnorkelSession
session = SnorkelSession()

import sys
sys.path.append(os.environ['SNORKELHOME'] + '/tutorials/tables/')

from snorkel.models import candidate_subclass
Part_Polarity = candidate_subclass('Part_Polarity', ['part','polarity'])

from snorkel.models import CandidateSet
train = session.query(CandidateSet).filter(
    CandidateSet.name == 'Hardware Training Candidates').one()

### Define LFs

In [2]:
from snorkel.fast_annotations import LabelManager
from snorkel.lf_helpers import *
import re
label_manager = LabelManager()

LFs = []

def LF_complement_phrase_part(c):
    return -1 if overlap(['complement','complementary'], 
                         get_phrase_ngrams(c.part)) else 0
LFs.append(LF_complement_phrase_part)

def LF_complement_phrase_polarity(c):
    return -1 if overlap(['complement','complementary'], 
                         get_phrase_ngrams(c.polarity)) else 0
LFs.append(LF_complement_phrase_polarity)

def LF_complement_neighbor_polarity(c):
    return -1 if overlap(['complement','complementary'], 
                         get_neighbor_phrase_ngrams(c.polarity)) else 0
LFs.append(LF_complement_neighbor_polarity)

def LF_top_mark_col_part(c):
    return -1 if overlap(['top','mark'],
                         get_col_ngrams(c.part)) else 0
LFs.append(LF_top_mark_col_part)

def LF_endswith_D_part(c):
    return -1 if c.part.get_span().endswith('D') else 0
LFs.append(LF_endswith_D_part)

def LF_default_positive(c):
    return 1 if not overlap(['complement','complementary'],
                            chain.from_iterable([
                            get_phrase_ngrams(c.part),
                            get_phrase_ngrams(c.polarity),
                            get_neighbor_phrase_ngrams(c.polarity)])) else 0
LFs.append(LF_default_positive)

In [6]:
from snorkel.models import Candidate
from hardware_utils import entity_to_candidates

candidates = session.query(Candidate).all()
print len(candidates)
# matches = entity_to_candidates((u'SIEMS01215-1', u'BC856', u'NPN'), candidates)
# c = matches[0]
c = candidates[0]
print candidates[0]

print LF_complement_phrase_part(c)
print LF_complement_phrase_polarity(c)
print LF_complement_neighbor_polarity(c)
print LF_top_mark_col_part(c)
print LF_endswith_D_part(c)
print LF_default_positive(c)

114067
Part_Polarity(ImplicitSpan("BC807-25", parent=6783, words=[0,0], position=[1]), Span("NPN", parent=6804, chars=[26,28], words=[2,2]))
0
-1
0
0
0
0


### Apply LFs

In [7]:
%time L_train = label_manager.create(session, train, 'LF Labels', f=LFs)
L_train

[========================================] 100%
Bulk upserting 552282 annotations...
Done.
Loading sparse Label matrix...
CPU times: user 15min 32s, sys: 2min, total: 17min 32s
Wall time: 17min 50s


<92047x6 sparse matrix of type '<type 'numpy.float64'>'
	with 92282 stored elements in Compressed Sparse Row format>

### Assess LF accuracy

In [8]:
train_gold = session.query(CandidateSet).filter(
    CandidateSet.name == 'Hardware Training Candidates -- Gold').one()
%time L_train.lf_stats(train_gold)

[========================================] 100%
CPU times: user 1.89 s, sys: 54.6 ms, total: 1.94 s
Wall time: 1.95 s


,accuracy,conflicts,coverage,j,overlaps
LF_complement_phrase_part,0,0.000000,0.000043,0,0.000000
LF_complement_phrase_polarity,1,0.000000,0.060632,1,0.000000
LF_complement_neighbor_polarity,0,0.000000,0.000500,2,0.000000
LF_top_mark_col_part,0,0.002553,0.002553,3,0.002553
LF_endswith_D_part,NaN,0.000000,0.000000,4,0.000000
LF_default_positive,1,0.002553,0.938825,5,0.002553


In [9]:
# If necessary
import os
os.system('cp snorkel.db snorkel.db\ features');

## Learn and Evaluate

In [ ]:
# If necessary:
import os
os.remove('snorkel.db');
os.system('cp snorkel.db\ features snorkel.db');

from snorkel import SnorkelSession
session = SnorkelSession()

import sys
sys.path.append(os.environ['SNORKELHOME'] + '/tutorials/tables/')

from snorkel.models import candidate_subclass
Part_Polarity = candidate_subclass('Part_Polarity', ['part','polarity'])

from snorkel.models import CandidateSet
train = session.query(CandidateSet).filter(
    CandidateSet.name == 'Hardware Training Candidates').one()
dev = session.query(CandidateSet).filter(
    CandidateSet.name == 'Hardware Development Candidates').one()

from snorkel.annotations import FeatureManager, LabelManager
feature_manager = FeatureManager()
%time F_train = feature_manager.load(session, train, 'Train Features')
%time F_dev = feature_manager.load(session, dev, 'Train Features')

label_manager = LabelManager()
%time L_train = label_manager.load(session, train, 'LF Labels')

In [ ]:
from snorkel.learning import NaiveBayes

gen_model = NaiveBayes()
gen_model.train(L_train, n_iter=15000, rate=1e-2)
%time gen_model.save(session, 'Generative Params')
train_marginals = gen_model.marginals(L_train)

In [ ]:
from snorkel.learning import LogReg

disc_model = LogReg()
disc_model.train(F_train, train_marginals, n_iter=2000, rate=1e-4)
%time disc_model.save(session, "Discriminative Params")

In [ ]:
train_gold = session.query(CandidateSet).filter(
    CandidateSet.name == 'Hardware Training Candidates -- Gold').one()

In [ ]:
dev_gold = session.query(CandidateSet).filter(
    CandidateSet.name == 'Hardware Development Candidates -- Gold').one()

In [ ]:
from snorkel.models import CandidateSet
from snorkel.annotations import LabelManager
label_manager = LabelManager()
L_dev = label_manager.load(session, dev, 'Hardware Development Labels -- Gold')

In [ ]:
tp, fp, tn, fn = disc_model.score(F_dev, L_dev, dev_gold)

In [ ]:
from snorkel.models import Corpus
from hardware_utils import entity_level_f1
import os

gold_file = os.environ['SNORKELHOME'] + '/tutorials/tables/data/hardware/hardware_gold.csv'
corpus = session.query(Corpus).filter(Corpus.name == 'Hardware Development').one()
(TP, FP, FN) = entity_level_f1(tp, fp, tn, fn, gold_file, corpus, 'stg_temp_max')

In [ ]:
from pprint import pprint
FN_list = sorted(list(FN))
pprint(FN_list)

In [ ]:
from hardware_utils import entity_to_candidates

entity = FN_list[0]
print entity
print

matches = entity_to_candidates(entity, fn)
print "# Matches: %d" % len(matches)
candidate = matches[0]
print candidate
print

print disc_model.get_candidate_score(candidate, F_dev)
print
pprint(disc_model.get_candidate_feature_weights(candidate, F_dev))

In [ ]:
from hardware_utils import part_error_analysis
part_error_analysis(candidate)

In [ ]:
import os
os.system('cp snorkel.db snorkel.db\ final');